In [1]:
import sys
sys.path
import torch
import torch.nn as nn
import torch.optim as optim

import nibabel as nib

/home/warrior/miniconda3/envs/dl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2
from model.uunet_3d import UNet

In [3]:
n1 = 64
filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]
filters

[64, 128, 256, 512, 1024]

In [4]:
model = UNet(filters=filters)
device = torch.device("cpu")
print(device)
#loss = nn.CrossEntropyLoss()
loss = nn.BCEWithLogitsLoss()

model.to(device)
model.eval()
print('')

cpu



# DATA
### Input _ [batch * channel(# of channels of each image) * depth(# of frames) * height * width].

In [5]:
nii_file  = nib.load('data/hippo_train.nii.gz')
nii_data = nii_file.get_fdata()
print(type(nii_data))
print('Height, Width, N_images, Channels')
print(nii_data.shape)

<class 'numpy.ndarray'>
Height, Width, N_images, Channels
(32, 32, 32)


In [6]:
x = torch.from_numpy(nii_data)
print(x.shape)
print(x.dtype)

torch.Size([32, 32, 32])
torch.float64


In [7]:
x = x.permute(2, 0, 1)
print(x.shape)
print(x.dtype)

torch.Size([32, 32, 32])
torch.float64


In [8]:
x = torch.unsqueeze(x, dim=0)
print(x.shape)
print(x.dtype)

torch.Size([1, 32, 32, 32])
torch.float64


In [9]:
x = torch.unsqueeze(x, dim=0)
print(x.shape)
print(x.dtype)

torch.Size([1, 1, 32, 32, 32])
torch.float64


In [10]:
y = model(x.float())

INPUT: torch.Size([1, 1, 32, 32, 32])
ENCODER
->e1 torch.Size([1, 64, 32, 32, 32])

!!e2 torch.Size([1, 64, 16, 16, 16])
->e2 torch.Size([1, 128, 16, 16, 16])

!!e3 torch.Size([1, 128, 8, 8, 8])
->e3 torch.Size([1, 256, 8, 8, 8])

!!e4 torch.Size([1, 256, 4, 4, 4])
->e4 torch.Size([1, 512, 4, 4, 4])

DECODER
^d3 torch.Size([1, 256, 8, 8, 8])
[]d3 torch.Size([1, 512, 8, 8, 8])
->d3 torch.Size([1, 256, 8, 8, 8])

^d2 torch.Size([1, 128, 16, 16, 16])
[]d2 torch.Size([1, 256, 16, 16, 16])
->d2 torch.Size([1, 128, 16, 16, 16])

^d1 torch.Size([1, 64, 32, 32, 32])
[]d1 torch.Size([1, 128, 16, 16, 16])
->d1 torch.Size([1, 64, 32, 32, 32])
1x1out torch.Size([1, 1, 32, 32, 32])


In [11]:
#print(len(y))
#y[0].shape

In [12]:
#torch.save(y[0], 'prediction/tensor_image.pth')